<a href="https://colab.research.google.com/github/javst/MachineVision/blob/main/CIRAR10Study.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

batchsz = 32
cifar_train = datasets.CIFAR10('cifar',True,transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
]),download=True)

cifar_train = DataLoader(cifar_train,batch_size=batchsz,shuffle=True)
cifar_test = datasets.CIFAR10('cifar',True,transform=transforms.Compose([
    transforms.Resize((32,32)),
    transforms.ToTensor()
]),download=True)

cifar_test = DataLoader(cifar_test,batch_size=batchsz,shuffle=True)

x,label = iter(cifar_train).next()
print('x:',x.shape,'label',label.shape)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting cifar/cifar-10-python.tar.gz to cifar
Files already downloaded and verified
x: torch.Size([32, 3, 32, 32]) label torch.Size([32])


In [ ]:
import torch
from torch import nn
from torch.nn import functional as F
class Lenet5(nn.Module):
    #for cifar 10 dataset
    def __init__(self):
        super(Lenet5,self).__init__()
        self.conv_unit = nn.Sequential(
            #x:[b,3,32,32]=>[b,6,]
            nn.Conv2d(3,6,kernel_size=5,stride=1,padding=0),
            nn.AvgPool2d(kernel_size=2,stride=2,padding=0),
            #
            nn.Conv2d(6,16,kernel_size=5,stride=1,padding=0),
            nn.AvgPool2d(kernel_size=2,stride=2,padding=0),
            #
            
        )
        #flatten
        #fcunitl
        self.fc_unit = nn.Sequential(
            nn.Linear(16*5*5,120),
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )

        #self.criteon = nn.CrossEntropyLoss()
        
    def forward(self,x):
        batchsz=x.size(0)
        x = self.conv_unit(x)
        b=x.view(batchsz,16*5*5)
        logits = self.fc_unit(b)
        return logits
        #pred = F.softmax(logits,dim=1)
        #loss = self.criteon(logits,y)

In [ ]:
from torch import optim
device = torch.device('cuda')
model = Lenet5().to(device)
criteon = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=1e-3)
for epoch in range(1000):
    for batchidx,(x,label) in enumerate(cifar_train):
        x,label = x.to(device),label.to(device)
        logits = model(x)
        loss = criteon(logits,label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if batchidx%10 == 0 :
           print(epoch,loss.item())
        
    
     
        
        

流式输出内容被截断，只能显示最后 5000 行内容。
21 1.1475335359573364
21 1.1384071111679077
21 0.8993129134178162
21 0.8119159936904907
21 0.544325053691864
21 0.6022028923034668
21 1.2103859186172485
21 0.6229750514030457
21 1.1275997161865234
21 0.8988017439842224
21 0.5682148933410645
21 0.7009329795837402
21 1.2462592124938965
21 0.8801992535591125
21 1.0750185251235962
21 1.2328245639801025
21 0.5841832160949707
21 0.8993752598762512
21 0.9942175149917603
21 1.0902132987976074
21 1.2116107940673828
21 0.8440884351730347
21 0.8387337923049927
21 1.0123381614685059
21 0.7599954605102539
21 1.2165380716323853
21 0.7122926712036133
21 0.7384500503540039
21 1.0460511445999146
21 1.1995222568511963
21 0.5544605851173401
21 0.8417655229568481
21 0.9142579436302185
21 0.9038827419281006
21 0.6482408046722412
21 1.2007932662963867
21 0.9290465116500854
21 0.8637052774429321
21 0.503804087638855
21 0.5612230896949768
21 0.799619197845459
21 0.7554899454116821
21 0.9514867663383484
21 0.7918997406959534
21 0.776

In [ ]:
total_correct = 0
total_num =0
with torch.no_grad():
  for x,label in cifar_test:
    x,label = x.to(device),label.to(device)
    logits = model(x)
    pred = logits.argmax(dim = 1)
    total_correct +=torch.eq(pred,label).float().sum()
    total_num +=x.size(0)
acc = total_corretc/total_num
print(epoch,acc)